In [ ]:
# -*- coding: utf-8 -*-

"""
Project: RRI Model Parameter Calibration via Browser-based Claude
Description:
    This script controls a browser-based Claude (via Selenium/Undetected-Chromedriver)
    to perform automatic iterative optimization of RRI model parameters.
    Note: The Claude interface may update, so selectors might need adjustment based on actual UI.
    
    Overall Workflow:
    1.  Launch Chrome browser and navigate to Claude.ai.
    2.  User manually logs into their Claude account.
    3.  The script reads an initial prompt and sends it to Claude to obtain a set of initial parameters.
    4.  The script then runs the RRI hydrological model (Fortran .exe) with these parameters to evaluate their performance.
    5.  The evaluation results (specifically, NSE scores) are fed back to Claude to request a new, improved set of parameters for the next iteration.
    6.  This process (sending parameters, running model, feeding back results) loops until the maximum number of optimization rounds is reached.

Important Notes: 
    - You need to modify `CHROMEDRIVER_PATH` to your local chromedriver executable path.
    - If the Claude interface language is not Japanese, you might need to adjust the `aria-label` values in CSS Selectors (e.g., from "応答を停止" to "Stop generating").
"""

import os        
import re        
import ast      
import time      
import random    
import subprocess 
import numpy as np 
import pandas as pd 
from osgeo import gdal 

# Selenium related imports for browser automation
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException 
import undetected_chromedriver as uc 

# ==========================================
# 1. Global Configuration & Path Settings
# ==========================================

# 1.1 File Path Configuration
PATHS = {
    'dem': "topo/dem_250.tif",        # Path to the Digital Elevation Model (DEM) GeoTIFF file.
    'out_folder': "out",              # Directory where the raw RRI model output files will be saved.
    'result_folder': "results",       # Directory for processed results and intermediate files.
    'out_prefix': "qr",               # Prefix for RRI model output files (e.g., qr_1.out, qr_2.out).
    'rri_exe': "0_rri_1_4_2_6.exe",   # Path to the RRI hydrological model executable (compiled Fortran program).
    'rri_input': "RRI_Input.txt",     # Path to the RRI model's input configuration file.
    'obs_data': "./results/obsQ.xlsx",# Path to the Excel file containing observed discharge (flow) data.
    'rain_folder': "./rain",          # Directory containing rainfall input data files for the RRI model.
    'prompt_file': "Prompt.txt"       # Path to the text file storing the initial prompt for Claude.
}

# 1.2 Browser Driver Path 
CHROMEDRIVER_PATH = r"./chromedriver.exe" # User-specific path to the ChromeDriver executable.

# 1.3 Optimization Settings
MAX_ROUNDS = 30        # Maximum number of optimization/dialogue rounds with Claude.
PAUSE_SECONDS = 5     # Cooldown time (in seconds) after each optimization round to avoid overwhelming Claude or rate limits.

# 1.4 Observation Point Coordinates (Longitude, Latitude)
LOCATIONS = [
    (136.16375, 36.0081) # A list of (longitude, latitude) tuples for points where simulated data will be extracted.
]

# 1.5 Rainfall Event List (Filename, Simulation Duration in Hours)
RAIN_EVENTS = [
    ('rain_merged_precipitation_20040717_to_20040723.dat', 168), # Tuple: rain data filename and corresponding simulation duration.
    ('rain_merged_precipitation_20041019_to_20041022.dat', 81),
    ('rain_merged_precipitation_20050630_to_20050709.dat', 240),
    ('rain_merged_precipitation_20110707_to_20110711.dat', 120),
    ('rain_merged_precipitation_20110919_to_20110926.dat', 192),
    ('rain_merged_precipitation_20171021_to_20171028.dat', 192),
    ('rain_merged_precipitation_20180704_to_20180711.dat', 192)
]

# 1.6 Fixed Parameters and Initial Template
FIXED_PARAMS = {
    'ksv_1': 0.0000, 'ksv_2': 0.0000, # Parameters that remain constant during the optimization process.
    'faif_1': 0.0000, 'faif_2': 0.0000
}

INITIAL_PARAMS = {
    'ns_river': 0.05,                   # Initial values for the RRI model parameters that will be optimized.
    'ns_slope_1': 0.4, 'soildepth_1': 1.0, 'gammaa_1': 0.4, 'gammam_1': 0.1, 'ka_1': 0.1, 'beta_1': 8.0,
    'ns_slope_2': 0.4, 'soildepth_2': 1.0, 'gammaa_2': 0.4, 'gammam_2': 0.1, 'ka_2': 0.1, 'beta_2': 8.0
}
INITIAL_PARAMS.update(FIXED_PARAMS) # Combine initial optimizable parameters with fixed ones.

# ==========================================
# 2. Browser Automation Functions (Selenium Control)
# ==========================================

def create_claude():
    """
    Creates and initializes a Chrome browser instance using undetected_chromedriver
    for interacting with Claude.ai. It includes anti-detection measures and
    prompts the user for manual login.
    
    Returns:
        driver: An initialized Selenium WebDriver object for Chrome.
    """
    options = uc.ChromeOptions()
    # Spoof User-Agent to mimic a regular browser.
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
    options.add_argument('--disable-blink-features=AutomationControlled') # Prevents detection as an automated tool.
    options.add_argument('--no-sandbox') # Required for running Chrome in some environments (e.g., Docker).
    options.add_argument('--disable-dev-shm-usage') # Workaround for Docker environments.
    
    print(f"Launching browser, driver path: {CHROMEDRIVER_PATH}")
    # Initialize the undetected_chromedriver with specified service and options.
    driver = uc.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
    
    # Further hide webdriver properties to prevent detection.
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    driver.get("https://claude.ai/new") # Navigate to Claude's new chat interface.
    
    print("\n" + "="*50)
    print("【Attention】Browser has been opened.")
    print("1. Please manually log in to your Claude account in the browser.")
    print("2. After successful login and seeing the chat interface, press Enter below to continue...")
    print("="*50 + "\n")
    input(">>> After logging in, press Enter to continue: ") # Waits for user to confirm manual login.
    
    time.sleep(2) # Small delay after user input.
    return driver

def wait_claude(driver):
    """
    Waits for Claude to finish generating its response.
    It works by detecting the presence and then disappearance of a "stop generating" button.
    
    Args:
        driver: The Selenium WebDriver object.
    
    Raises:
        TimeoutException: If Claude's response takes longer than 10 minutes.
    """
    # Note: If your Claude interface is in English, change 'aria-label' to "Stop generating" or similar.
    stop_button_selector = 'button[aria-label="応答を停止"]' # CSS selector for the stop generating button.
    
    # Phase 1: Wait for the "stop" button to appear (confirming generation has started).
    try:
        WebDriverWait(driver, 20).until( # Wait up to 20 seconds.
            EC.presence_of_element_located((By.CSS_SELECTOR, stop_button_selector))
        )
    except TimeoutException:
        print("Warning: 'Stop generating' button not detected. Claude might not have started generating or responded extremely quickly.")
        return # Continue if button doesn't appear, assuming response was fast.

    # Phase 2: Wait for the "stop" button to disappear (confirming generation has ended).
    try:
        # Wait up to 600 seconds (10 minutes) for Claude to finish.
        WebDriverWait(driver, 600).until_not(
            EC.presence_of_element_located((By.CSS_SELECTOR, stop_button_selector))
        )
    except TimeoutException:
        raise TimeoutException("Waiting for Claude's response timed out (exceeded 10 minutes)!")

def prompting_claude(prompt, driver):
    """
    Sends a given prompt to Claude and retrieves its latest response.
    
    Args:
        prompt (str): The text content to send to Claude.
        driver: The Selenium WebDriver object.
        
    Returns:
        str: The text content of Claude's reply, or None if an error occurs.
    """
    try:
        # 1. Locate the input textbox and enter the prompt text.
        input_selector = 'div[role="textbox"]' # CSS selector for the chat input box.
        input_element = WebDriverWait(driver, 20).until( # Wait for the input element to be present.
            EC.presence_of_element_located((By.CSS_SELECTOR, input_selector))
        )
        input_element.click() # Click to ensure the element is in focus.
        input_element.clear() # Clear any pre-existing text.
        
        # Simulate human-like typing effect (to avoid bot detection).
        # Adjust sleep time if the prompt is very long.
        for char in prompt:
            input_element.send_keys(char) # Type character by character.
            time.sleep(random.uniform(0.001, 0.005)) # Short random delay between characters.
        
        # 2. Locate and click the send button.
        # Note: If the interface is in English, change 'aria-label' to "Send Message".
        send_button_selector = 'button[aria-label="メッセージを送信"]:not([disabled])' # CSS selector for the send button.
        send_button = WebDriverWait(driver, 10).until( # Wait for the send button to be clickable.
            EC.element_to_be_clickable((By.CSS_SELECTOR, send_button_selector))
        )
        driver.execute_script("arguments[0].click();", send_button) # Use JavaScript click to ensure reliability.
        
        # 3. Wait for Claude to finish generating its response.
        wait_claude(driver)
        
        # 4. Retrieve the latest response.
        # 'data-test-render-count="1"' often points to the latest rendered message,
        # but Claude's UI updates might make this selector unstable.
        final_response_selector = 'div[data-test-render-count="1"] .standard-markdown'
        
        try:
            final_response_element = WebDriverWait(driver, 30).until( # Wait for the final response element.
                EC.presence_of_element_located((By.CSS_SELECTOR, final_response_selector))
            )
        except TimeoutException:
            print("Warning: Could not locate latest response using data-test-render-count, trying fallback method...")
            # Fallback method: Get all response bubbles and take the last one.
            all_responses = driver.find_elements(By.CSS_SELECTOR, 'div[data-is-response="true"]')
            if all_responses:
                return all_responses[-1].text # Return the text of the last response bubble.
            else:
                return None # No responses found.

        return final_response_element.text # Return the text content of the latest response.

    except Exception as e:
        print(f"An error occurred during Claude interaction: {e}")
        return None

def close_claude(driver):
    """
    Closes the browser window and quits the WebDriver session to release resources.
    
    Args:
        driver: The Selenium WebDriver object.
    """
    if driver: # Check if the driver object exists.
        try:
            driver.close() # Close the current window.
            driver.quit()  # Quit the browser application.
        except:
            pass # Ignore errors during closing, as it might already be closed.

# ==========================================
# 3. LLM Management Class (Connecting RRI with Browser)
# ==========================================

class LLM_Manager:
    """
    Manages the interaction logic with the Large Language Model (LLM) via the browser,
    including constructing prompts, calling browser automation functions, and parsing
    the LLM's responses.
    """
    def __init__(self, prompt_path, driver):
        """
        Initializes the LLM_Manager.
        
        Args:
            prompt_path (str): Path to the initial prompt file.
            driver: The Selenium WebDriver object.
        """
        self.prompt_path = prompt_path
        self.driver = driver
        self.base_prompt = self._read_prompt() # Reads the initial prompt from file.
        self.history = [] # Stores interaction history, though not explicitly used for chat history in prompt building in this version.

    def _read_prompt(self):
        """
        Reads the initial prompt content from the specified file path.
        
        Returns:
            str: The content of the prompt file, or an empty string if not found.
        """
        if not os.path.exists(self.prompt_path):
            print(f"Error: {self.prompt_path} not found.")
            return ""
        with open(self.prompt_path, 'r', encoding='utf-8') as f:
            return f.read()

    def parse_response(self, response_text):
        """
        Parses the text response from the LLM to extract a list of parameter sets.
        It supports extracting Python list formats from Markdown code blocks
        (e.g., ```json[...]``` or ```python[...]```) or from plain text.
        
        Args:
            response_text (str): The raw text response received from Claude.
            
        Returns:
            list: A list of lists, where each inner list represents a set of parameters.
                  Returns an empty list if parsing fails or the format is incorrect.
        """
        if not response_text:
            return []
        
        try:
            # 1. Attempt to extract content within ```json or ```python code blocks.
            match = re.search(r"```(?:json|python)?(.*?)```", response_text, re.DOTALL)
            if match:
                clean_text = match.group(1).strip()
            else:
                # 2. If no code block, try to find the outermost [[ ... ]] for a list of lists.
                start = response_text.find('[[')
                end = response_text.rfind(']]') + 2
                if start != -1 and end != -1:
                    clean_text = response_text[start:end]
                else:
                    clean_text = response_text.strip() # If not found, take the whole text.
            
            # 3. Safely evaluate the extracted string into a Python object.
            params_list = ast.literal_eval(clean_text)
            
            # Validate that the parsed object is a non-empty list of lists.
            if isinstance(params_list, list) and len(params_list) > 0 and isinstance(params_list[0], list):
                return params_list
            else:
                print("Parsing failed: Response content is not in a 2D list format.")
                return []

        except Exception as e:
            print(f"Error parsing LLM response: {e}")
            print(f"Original content snippet: {response_text[:100]}...") # Print a snippet for debugging.
            return []

    def get_next_params(self, last_formatted_list=None, last_params_values=None):
        """
        Constructs the prompt for Claude and sends it to get the next set of parameters.
        For the first round, it sends the base prompt. For subsequent rounds, it
        includes feedback from the previous model evaluation.
        
        Args:
            last_formatted_list (list, optional): A list of NSE scores from the previous round,
                                                 formatted for readability. Defaults to None for the first round.
            last_params_values (list, optional): A list of parameter sets used in the previous round.
                                                Defaults to None for the first round.
                                                
        Returns:
            list: A list of new parameter sets proposed by Claude.
        """
        
        if last_formatted_list is None:
            # First round: Send only the base prompt to get initial parameter suggestions.
            current_prompt = self.base_prompt
        else:
            # Subsequent rounds: Append feedback data to the base prompt.
            feedback_str = "\n\n### Feedback from Previous Round ###\n"
            feedback_str += "Please analyze the following parameter combinations and their corresponding NSE scores (higher scores are better):\n"
            
            # Iterate through the previous round's scores and parameter sets to build feedback.
            for i, (score, params) in enumerate(zip(last_formatted_list, last_params_values)):
                feedback_str += f"Set {i+1}: Parameters={params} -> NSE Score={score}\n"
            
            feedback_str += "\nBased on the above results, please generate a new list of better candidate parameters.\n"
            feedback_str += "Please **only** output the list format, without any explanatory text. Format example: [[0.1, ...], [0.2, ...]]"
            
            # Concatenate the base prompt and the feedback string.
            current_prompt = self.base_prompt + feedback_str

        print(f"Sending Prompt to Claude (length: {len(current_prompt)} characters)...")
        # Call the browser automation function to send the prompt and get Claude's response.
        response_text = prompting_claude(current_prompt, self.driver)
        
        if response_text:
            print("\n--- Claude's Response ---")
            # Print the first 200 characters of the response for debugging/monitoring.
            print(response_text[:200] + "..." if len(response_text)>200 else response_text)
            print("-------------------\n")
            
        # Parse Claude's response to extract the new parameter values.
        new_values = self.parse_response(response_text)
        return new_values

# ==========================================
# 4. RRI Model Helper Functions (File I/O & Calculation)
# ==========================================

def format_fortran_line(par_name, params_dict, num_landuse):
    """
    Formats a parameter line for the RRI model's Fortran input file.
    It takes a base parameter name and retrieves values for different land uses
    from a dictionary, formatting them as double-precision (e.g., "0.123456d0").
    
    Args:
        par_name (str): The base name of the parameter (e.g., "# ns_slope").
        params_dict (dict): A dictionary containing all RRI model parameters.
        num_landuse (int): The number of land-use types in the RRI model.
        
    Returns:
        str: The formatted string for the RRI_Input.txt file, or an empty string if error.
    """
    values = []
    base_name = par_name[2:].strip() # Remove "# " prefix.
    for j in range(num_landuse):
        param_key = f"{base_name}_{j+1}" # Construct parameter key (e.g., "ns_slope_1").
        if param_key in params_dict:
            values.append(f"{params_dict[param_key]:.6f}d0") # Format as double-precision Fortran literal.
        else:
            print(f"Error: Parameter {param_key} not found in the provided dictionary.")
            return ""
    return '\t'.join(values) + f"      # {base_name}\n" # Join values with tabs and add comment.

def modify_rri_input_file(params_dict, rain_file_path, duration_hours):
    """
    Modifies the `RRI_Input.txt` file with new rainfall data path, simulation duration,
    and updated RRI model parameters from `params_dict`.
    
    Args:
        params_dict (dict): A dictionary of RRI model parameters to be updated.
        rain_file_path (str): The filename of the rainfall data to use for this simulation.
        duration_hours (int): The total simulation duration in hours.
        
    Returns:
        bool: True if the file was successfully modified, False otherwise.
    """
    try:
        with open(PATHS['rri_input'], 'r') as f:
            lines = f.readlines() # Read all lines from the RRI input template file.
    except FileNotFoundError:
        print(f"Error: {PATHS['rri_input']} not found.")
        return False
    
    full_rain_path = os.path.join(PATHS['rain_folder'], rain_file_path) # Construct full path for rain data.
    
    # Modify specific lines in the RRI_Input.txt file.
    lines[2] = f"{full_rain_path}\n" # Update rainfall data path.
    lines[9] = f"{duration_hours}    # lasth(hour)\n" # Update simulation end time.
    lines[12] = f"{duration_hours}    # outnum [-]\n" # Update output steps (often matches duration).

    num_landuse = None
    # Find the number of land-use types from the input file.
    for i, line in enumerate(lines):
        if "# num_of_landuse" in line:
            num_landuse = int(lines[i].split('#')[0].strip())
            break
            
    if num_landuse is None:
        print("Error: Could not find 'num_of_landuse' in RRI_Input.txt.")
        return False

    # List of parameters that need multi-landuse formatting.
    target_params = ["# ns_slope", "# soildepth", "# gammaa", "# ksv", 
                     "# faif", "# ka", "# gammam", "# beta"]
    
    # Iterate through lines to update parameter values.
    for i, line in enumerate(lines):
        if "# ns_river" in line and 'ns_river' in params_dict:
            lines[i] = f"{params_dict['ns_river']:.6f}d0     # ns_river\n" # Update single-value parameter.
        else:
            for tag in target_params: # For multi-landuse parameters.
                if tag in line:
                    lines[i] = format_fortran_line(tag, params_dict, num_landuse) # Use helper to format.
                    break
    
    # Write the modified lines back to the RRI_Input.txt file.
    with open(PATHS['rri_input'], 'w') as f:
        f.writelines(lines)
    return True

def get_raster_info(tif_path):
    """
    Reads essential metadata (dimensions, geotransform) from a GeoTIFF file.
    
    Args:
        tif_path (str): Path to the GeoTIFF file.
        
    Returns:
        dict: A dictionary containing 'cols' (number of columns), 'rows' (number of rows),
              and 'geotransform' (GDAL geotransform tuple).
              
    Raises:
        Exception: If the GeoTIFF file cannot be opened.
    """
    ds = gdal.Open(tif_path) # Open the GeoTIFF dataset.
    if ds is None: raise Exception(f"Cannot open {tif_path}")
    return {'cols': ds.RasterXSize, 'rows': ds.RasterYSize, 'geotransform': ds.GetGeoTransform()}

def lonlat_to_xy(lon, lat, geotransform):
    """
    Converts geographic coordinates (longitude, latitude) to raster pixel coordinates (column, row).
    
    Args:
        lon (float): Longitude.
        lat (float): Latitude.
        geotransform (tuple): GDAL geotransform tuple (top-left x, w-e pixel resolution,
                              row rotation, top-left y, n-s pixel resolution, col rotation).
                              
    Returns:
        tuple: (x, y) where x is column index and y is row index.
    """
    # Inverse geotransform formula to convert geo coordinates to pixel coordinates.
    x = int((lon - geotransform[0]) / geotransform[1])
    y = int((lat - geotransform[3]) / geotransform[5])
    return x, y

def read_out_file(out_file):
    """
    Reads a numerical matrix from a plain text .out file (RRI output).
    
    Args:
        out_file (str): Path to the .out file.
        
    Returns:
        numpy.ndarray: A NumPy array representing the data matrix, or an empty array if loading fails.
    """
    try: return np.loadtxt(out_file) # Use numpy to load data efficiently.
    except: return np.array([]) # Return empty array on error.

def extract_time_series(lon, lat, tif_path, out_folder, out_prefix):
    """
    Extracts a time series of values for a specific geographic coordinate
    from a sequence of RRI model output files (e.g., qr_1.out, qr_2.out).
    
    Args:
        lon (float): Longitude of the observation point.
        lat (float): Latitude of the observation point.
        tif_path (str): Path to the DEM GeoTIFF file (for coordinate conversion).
        out_folder (str): Directory where RRI output files are stored.
        out_prefix (str): Prefix of the RRI output files.
        
    Returns:
        dict: A dictionary containing 'time_steps' and 'values' lists,
              or None if raster info cannot be obtained.
    """
    try: raster_info = get_raster_info(tif_path)
    except: return None # Return None if DEM info can't be read.
    x, y = lonlat_to_xy(lon, lat, raster_info['geotransform']) # Convert geo to pixel coordinates.
    
    # Regex pattern to match RRI output files and extract their time step number.
    pattern = re.compile(rf'{out_prefix}_(\d+)\.out')
    # List and sort all relevant output files by their time step.
    all_files = sorted([os.path.join(out_folder, f) for f in os.listdir(out_folder) if pattern.match(f)],
                       key=lambda x: int(pattern.search(os.path.basename(x)).group(1)))
    
    values = []
    steps = []
    for fpath in all_files:
        data = read_out_file(fpath) # Read data for each time step.
        # Check if data is valid and coordinates are within bounds.
        if len(data) > 0 and y < data.shape[0] and x < data.shape[1]:
            values.append(data[y, x]) # Extract value at the specified pixel.
        else:
            values.append(np.nan) # Append NaN if data is invalid or out of bounds.
        steps.append(int(pattern.search(os.path.basename(fpath)).group(1))) # Store the time step.
    return {'time_steps': steps, 'values': values}

def get_simulated_values():
    """
    Orchestrates the extraction of simulated time series data for all predefined
    observation locations from RRI model outputs and cleans up temporary files.
    
    Returns:
        numpy.ndarray: A NumPy array of simulated values (assuming a single observation point,
                       or concatenating if multiple, based on `results[0]['value']` structure).
                       Returns an empty array if no results are found.
    """
    os.makedirs(PATHS['result_folder'], exist_ok=True) # Ensure the results folder exists.
    results = []
    # Loop through all defined observation locations.
    for lon, lat in LOCATIONS:
        ts = extract_time_series(lon, lat, PATHS['dem'], PATHS['out_folder'], PATHS['out_prefix'])
        if ts: results.append(pd.DataFrame({'value': ts['values']})) # If successful, store as DataFrame.
    
    # Clean up the 'out' folder by removing temporary RRI output files.
    for f in os.listdir(PATHS['out_folder']):
        try: os.remove(os.path.join(PATHS['out_folder'], f))
        except: pass
        
    if not results: return np.array([]) # Return empty array if no simulation results were extracted.
    return results[0]['value'].values # Return values from the first (and typically only) observation point.

def update_params_dict(params_dict, new_values):
    """
    Updates a given RRI parameter dictionary with new values,
    skipping any parameters defined as 'fixed'.
    
    Args:
        params_dict (dict): The base dictionary of RRI parameters (e.g., INITIAL_PARAMS).
        new_values (list): A list of new parameter values received from the LLM.
        
    Returns:
        dict: A new dictionary with updated parameters, or None if the number of
              new values does not match the number of optimizable parameters.
    """
    # Keys that should not be updated (fixed parameters).
    skip_keys = ['gammaa_1', 'gammaa_2', 'ksv_1', 'ksv_2', 'faif_1', 'faif_2']
    # Identify keys that are eligible for update.
    update_keys = [key for key in params_dict.keys() if key not in skip_keys]
    
    if len(new_values) != len(update_keys):
        print(f"Warning: Parameter count mismatch. Expected {len(update_keys)} values, but received {len(new_values)}.")
        return None 
    
    updated_dict = params_dict.copy() # Create a copy to avoid modifying the original.
    # Assign new values to the optimizable parameters.
    for key, value in zip(update_keys, new_values):
        updated_dict[key] = value
    return updated_dict

def run_model_and_evaluate(params_dict):
    """
    Runs a complete RRI model evaluation cycle for a given set of parameters,
    covering multiple rainfall events, and calculates the Nash-Sutcliffe Efficiency (NSE) score.
    
    Args:
        params_dict (dict): The dictionary of RRI model parameters to use for the simulation.
        
    Returns:
        tuple: (average_NSE, list_of_individual_NSE_scores)
               average_NSE is the mean of valid NSE scores across all events.
               list_of_individual_NSE_scores contains NSE for each event.
               Returns -999.0 for NSE if calculation fails.
    """
    nse_list = []
    # Iterate through each defined rainfall event.
    for index, (rain_file, duration) in enumerate(RAIN_EVENTS):
        # Modify the RRI input file for the current event's rainfall and parameters.
        if not modify_rri_input_file(params_dict, rain_file, duration):
            nse_list.append(-999.0); continue # Append error code if input file modification fails.
        
        try:
            # Execute the RRI Fortran model executable.
            # `check=True` raises an exception for non-zero exit codes.
            # `stdout` and `stderr` are captured to avoid console clutter.
            subprocess.run([PATHS['rri_exe']], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except subprocess.CalledProcessError as e:
            print(f"    Exe execution failed for event {index+1}: {e.stderr.decode().strip()}")
            nse_list.append(-999.0); continue # Append error code if RRI execution fails.
        
        sim_data = get_simulated_values() # Extract simulated discharge data.
        
        try:
            # Read observed discharge data for the current event from the Excel file.
            obs_df = pd.read_excel(PATHS['obs_data'])
            obs_values = obs_df[index + 1].dropna().values # Get data for current event column, drop NaNs.
        except Exception as e:
            print(f"    Obs data read failed for event {index+1}: {e}")
            nse_list.append(-999.0); continue # Append error code if observed data reading fails.
            
        if len(sim_data) != len(obs_values) or len(obs_values) == 0:
            print("    Data length mismatch or no observed data. Cannot calculate NSE.")
            nse_list.append(-999.0); continue # Append error code if data lengths don't match or no obs data.
            
        # Calculate Nash-Sutcliffe Efficiency (NSE).
        mean_obs = np.mean(obs_values)
        denom = np.sum((obs_values - mean_obs) ** 2)
        # Avoid division by zero; if denominator is zero, NSE is undefined or typically set to a very low value.
        nse = 1 - np.sum((obs_values - sim_data) ** 2) / denom if denom != 0 else -999.0
        nse_list.append(nse)

    # Calculate the average NSE, excluding any error values (-999.0).
    valid_nses = [n for n in nse_list if n > -100] # Filter out error codes.
    avg_nse = np.mean(valid_nses) if valid_nses else -999.0 # Calculate mean if valid scores exist.
    return avg_nse, nse_list

# ==========================================
# 5. Main Program Entry
# ==========================================

if __name__ == "__main__":
    # The following commented-out block was used for initial testing
    # of the RRI model evaluation logic without involving the browser-based LLM.
    '''
    ## Original test, without using LLM in browser
    # new_values represents a batch of parameter sets that would typically come from the LLM.
    new_values = [
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.04, 2.51, 2.11],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.095, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.275, 0.185, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.065, 0.045, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.075, 0.055, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.12],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.49, 2.09],
    [0.10, 0.055, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.025, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.105, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.095, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.035, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.025, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.085, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.285, 0.195, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.275, 0.185, 0.09, 0.03, 0.065, 0.045, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.095, 0.03, 0.075, 0.055, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.52, 2.13],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.48, 2.08],
    [0.10, 0.05, 0.03, 0.278, 0.188, 0.092, 0.032, 0.072, 0.052, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.282, 0.192, 0.088, 0.028, 0.068, 0.048, 2.50, 2.10],
    [0.102, 0.052, 0.032, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.098, 0.048, 0.028, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.10],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.51, 2.11],
    [0.10, 0.05, 0.03, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.50, 2.09],
    [0.10, 0.05, 0.03, 0.281, 0.191, 0.091, 0.031, 0.071, 0.051, 2.51, 2.11],
    [0.10, 0.05, 0.03, 0.279, 0.189, 0.089, 0.029, 0.069, 0.049, 2.49, 2.09],
    [0.101, 0.051, 0.031, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.51, 2.11],
    [0.099, 0.049, 0.029, 0.28, 0.19, 0.09, 0.03, 0.07, 0.05, 2.49, 2.09]
    ]
    out_avg_nse = []        # List to store average NSE for each parameter set.
    out_detailed_nse = []   # List to store detailed NSE for each event for each parameter set.
    
    current_params = INITIAL_PARAMS.copy() # Start with initial parameters.
    
    print("Starting Batch Evaluation from LLM Parameters...")
    
    for ii, val_list in enumerate(new_values):
        print(f"\nProcessing Set {ii+1}/{len(new_values)}...")
        
        try:
            updated_dict = update_params_dict(current_params, val_list) # Update parameters for this set.
            
            avg_nse, detailed_nse = run_model_and_evaluate(updated_dict) # Run model and evaluate.
            
            out_avg_nse.append(avg_nse)        # Store results.
            out_detailed_nse.append(detailed_nse)
            
        except ValueError as e:
            print(f"Skipping Set {ii+1} due to error: {e}")
            out_avg_nse.append(-999.0)
            out_detailed_nse.append([])

    formatted_list = [round(num, 3) for num in out_avg_nse] # Format average NSE for output.
    
    print('\n================ RESULTS ================')
    print('Detailed NSE List per set:')
    print(out_detailed_nse)
    print('-----------------------------------------')
    print('Formatted Average NSE List (for LLM):')
    print(formatted_list)
    print('=========================================')
    ''' 
    
    driver = None # Initialize driver to None, so it can be safely closed in finally block.
    try:
        # 1. Launch the browser and prompt for manual login to Claude.
        driver = create_claude()
        
        # 2. Initialize the LLM_Manager, which handles communication with Claude.
        llm = LLM_Manager(PATHS['prompt_file'], driver)
        
        # 3. Main optimization loop.
        current_formatted_list = None # Stores NSE scores from the previous round, used as feedback for Claude.
        current_values_list = None    # Stores parameter sets from the previous round, used as feedback for Claude.
        best_overall_nse = -999.0     # Tracks the highest NSE score achieved across all rounds.
        best_overall_params = []      # Stores the parameter set that yielded the `best_overall_nse`.

        print(f"Starting optimization process. Maximum rounds: {MAX_ROUNDS}")

        for round_idx in range(MAX_ROUNDS):
            print(f"\n" + "="*40)
            print(f" ROUND {round_idx + 1} / {MAX_ROUNDS}")
            print(f"="*40)
            
            # 3.1 Get a batch of new candidate parameter sets from Claude.
            # Passes the previous round's performance (NSE scores and parameter values) as feedback.
            new_values_batch = llm.get_next_params(current_formatted_list, current_values_list)
            
            if not new_values_batch:
                print("Did not receive valid parameters from Claude, stopping loop.")
                break # Exit if Claude doesn't provide new parameters.
                
            print(f"LLM generated {len(new_values_batch)} sets of parameters, starting evaluation...")
            
            # Variables to store results for the current round.
            round_avg_scores = []
            round_detailed_scores = []
            
            # 3.2 Evaluate each parameter set in the batch.
            for i, val_list in enumerate(new_values_batch):
                print(f"  Eval Set {i+1}: {val_list}")
                
                # Update the base RRI parameter dictionary with the current candidate values.
                current_params = update_params_dict(INITIAL_PARAMS, val_list)
                if current_params is None:
                    round_avg_scores.append(-999.0) # Mark as error if update fails.
                    continue

                # Run the RRI model and calculate NSE for the updated parameters.
                avg_nse, detailed = run_model_and_evaluate(current_params)
                print(f"    -> Avg NSE: {avg_nse:.4f}")
                
                round_avg_scores.append(avg_nse)      # Store the average NSE for this parameter set.
                round_detailed_scores.append(detailed) # Store detailed NSE for this set.
                
                # Update the overall best performance if a better NSE is found.
                if avg_nse > best_overall_nse:
                    best_overall_nse = avg_nse
                    best_overall_params = val_list
                    print(f"    *** New best value found: {best_overall_nse:.4f} ***")

            # 3.3 Prepare data for the next round's feedback.
            current_values_list = new_values_batch # The parameters used in this round.
            # Format the average NSE scores to 3 decimal places for clearer feedback to Claude.
            current_formatted_list = [round(num, 3) for num in round_avg_scores]
            
            print(f"\nEnd of Round {round_idx + 1}.")
            print(f"Scores list for feedback: {current_formatted_list}")
            
            # Introduce a pause to avoid frequent requests to Claude.
            print(f"Waiting for {PAUSE_SECONDS} seconds...")
            time.sleep(PAUSE_SECONDS)

        # 4. Final output after the optimization loop completes.
        print("\n" + "="*40)
        print(" Optimization Finished ")
        print("="*40)
        print(f"Best NSE achieved: {best_overall_nse:.6f}")
        print(f"Best parameters: {best_overall_params}")
        
        # Save the best results to a file.
        with open('final_best_params.txt', 'w') as f:
            f.write(f"Best NSE: {best_overall_nse}\n")
            f.write(f"Params: {best_overall_params}\n")
            
    except Exception as e:
        print(f"An unhandled exception occurred: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging.
        
    finally:
        print("Closing browser...")
        close_claude(driver) # Ensure the browser is closed even if errors occur.